# Bike sharing in Washington
<em>Individual Project Python<em>
___

                              Master in Business Analytics & Big data
                                 Professor Juan Luis Cano Rodriguez
                                           Martin Hofbauer
                                              _________

## Libraries

In [68]:
from distributed import Client, progress
import dask
import dask.dataframe as dd
import dask.array as da
import numpy as np
import dask_ml
from dask_ml.preprocessing import Categorizer, DummyEncoder
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
from dask_ml.linear_model import LinearRegression
from dask_ml.metrics import mean_squared_error
from dask_ml.metrics import r2_score

In [58]:
client = Client()
client

C:\Users\Martin\Anaconda3\lib\site-packages\distributed\bokeh\core.py:57: UserWarning: 
Failed to start diagnostics server on port 8787. [WinError 10048] Normalerweise darf jede Socketadresse (Protokoll, Netzwerkadresse oder Anschluss) nur jeweils einmal verwendet werden
  warnings.warn('\n' + msg)


Client Scheduler: tcp://127.0.0.1:56691 Dashboard: http://127.0.0.1:56694/status,Cluster Workers: 4 Cores: 8 Memory: 17.10 GB


## Data Loading

In [59]:
hour_data = dd.read_csv(
    "https://gist.githubusercontent.com/geraldwal/b5a83f4c670abe0a662abce558e5d433/raw/bce4bbfc63355606e4503964e25798b5d2190b9b/hour%2520-%2520Python%2520Bike%2520Sharing",
    sep=",",
    parse_dates=["dteday"],
)

## Preparation for Baseline

### Renaming the Colums

In [60]:
new_columns = [
    "instant",
    "dteday",
    "season",
    "year",
    "month",
    "hour",
    "holiday",
    "weekday",
    "workingday",
    "weather",
    "temp",
    "atemp",
    "humidity",
    "windspeed",
    "casual",
    "registered",
    "count",
]
hour_data= hour_data.rename(columns=dict(zip(hour_data.columns, new_columns)))

In [32]:
hour_data = hour_data.drop(["dteday"], axis=1)

### Applying the Corrrect Datatypes and DummyEncoding

In [33]:
categs = ['season', 'year', 'month', 'hour', 'holiday', 'weekday', 'workingday', 'weather']

In [34]:
pipeline = make_pipeline(Categorizer(columns=categs), DummyEncoder(columns=categs))
hour_data_cl_hot_encoded = pipeline.fit_transform(hour_data)

### Splitting the Dataset

The intital Split function needed to be adapted a bit, including a .loc after the X and y. Furthermore, the trainsize needed to be reduced by 1 in order not to have rows included in both train and test.

In [35]:
def split_data(dataset, Target):
    X = dataset.loc[:, dataset.columns != Target]
    y = dataset.loc[:, Target]
    train_size = int(len(dataset) * 0.875)
    X_train, X_test, y_train, y_test = (
        X.loc[0:train_size -1],
        X.loc[train_size : len(dataset)],
        y.loc[0:train_size -1],
        y.loc[train_size : len(dataset)],
    )
    return X_train, X_test, y_train, y_test

In [36]:
x_train_reg, x_test_reg, y_train_reg, y_test_reg = split_data(hour_data_cl_hot_encoded, "registered")
x_train_casual, x_test_casual, y_train_casual, y_test_casual = split_data(hour_data_cl_hot_encoded, "casual")
x_train_count, x_test_count, y_train_count, y_test_count = split_data(hour_data_cl_hot_encoded, "count")

In [37]:
x_train_reg_drop, x_test_reg_drop = (
    x_train_reg.drop(["casual", "count"], axis=1),
    x_test_reg.drop(["casual", "count"], axis=1),
)

In [38]:
x_train_reg_arr, x_test_reg_arr, y_train_reg_arr, y_test_reg_arr = (
    x_train_reg_drop.values,
    x_test_reg_drop.values,
    y_train_reg.values,
    y_test_reg.values,
)

In [39]:
x_train_casual_drop, x_test_casual_drop = (
    x_train_casual.drop(["registered", "count"], axis=1),
    x_test_casual.drop(["registered", "count"], axis=1),
)

In [40]:
x_train_casual_arr, x_test_casual_arr, y_train_casual_arr, y_test_casual_arr = (
    x_train_casual_drop.values,
    x_test_casual_drop.values,
    y_train_casual.values,
    y_test_casual.values,
)

In [41]:
x_train_count_drop, x_test_count_drop = (
    x_train_count.drop(["casual", "registered"], axis=1),
    x_test_count.drop(["casual", "registered"], axis=1),
)

In [42]:
x_train_count_arr, x_test_count_arr, y_train_count_arr, y_test_count_arr = (
    x_train_count_drop.values,
    x_test_count_drop.values,
    y_train_count.values,
    y_test_count.values,
)

In [55]:
def score_lin(X_train, X_test, y_train, y_test):
    lm = LinearRegression()
    lm.fit(X_train, y_train)
    y_pred = lm.predict(X_test)
    print("Intercept:", lm.intercept_)
    print("Coefficients:", lm.coef_)
    print("Mean squared error (MSE): {:.2f}".format(mean_squared_error(y_test, y_pred)))
    print("Variance score (R2): {:.2f}".format(r2_score(y_test, y_pred)))
    return y_pred

In [56]:
##  baseline predictions and score for count
# train_dropped_target = x_train_count_arr.drop(["casual", "registered"], axis = 1)
# test_dropped_target = x_test_count_arr.drop(["casual", "registered"], axis = 1)
baseline_registered_count = score_lin(x_train_count_arr, x_test_count_arr, y_train_count_arr, y_test_count_arr)

C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  contains = index in indices
C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:595: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  sub[blockwise_token(i)] = blockwise_token(indices.index(index))


Intercept: -46.42112740023391
Coefficients: [ 6.43302601e+01  4.50575452e-03  1.17771970e+02  9.29013330e+01
 -8.26383575e+01 -3.01002803e+01 -1.75271798e+01  1.14335274e+01
 -2.06408918e+00  2.39771706e+01 -2.25634845e+01  2.64342305e+01
 -8.27196825e+00 -5.96714879e+00  7.66168075e+00  4.24776439e+00
  1.86606490e+01  6.22753287e+00 -9.64980103e+00  6.60016121e+00
  2.77375273e+01  5.40501318e+00 -1.21735800e+01 -1.46738021e+01
 -1.16708998e+02 -1.34093669e+02 -1.42242065e+02 -1.53280421e+02
 -1.56498566e+02 -1.40354809e+02 -8.32647284e+01  4.63709524e+01
  1.78730044e+02  3.88505442e+01 -1.20025965e+01  1.21106074e+01
  4.90852042e+01  4.50753598e+01  2.99070680e+01  3.75864220e+01
  9.76274690e+01  2.50836932e+02  2.23488940e+02  1.19593544e+02
  4.06950686e+01 -7.80853412e+00 -4.48572337e+01 -8.44972491e+01
  1.50743417e+01 -3.16770353e+00  8.74035944e+00 -3.51544206e+00
 -1.24139372e+00  1.63980441e+00  3.18344623e+00  3.50487695e+00
  6.81555993e+00  8.54607685e-01  3.27248246e+

C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  contains = index in indices


Mean squared error (MSE): 14814.57


ValueError: cannot convert float NaN to integer